In [1]:
## Get dependencies ##

import string
import math
import sys
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn
sys.path.append('..')
from GIR import *
import scipy as sp
import pickle
import time
import scipy as sp
from scipy import ndimage
from scipy import signal
import os
import statsmodels.api as sm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import glob
import requests
import ftplib
import PyPDF2
import io
import cmocean
import multiprocessing
import xarray as xr
import numpy as np
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import zarr
import gcsfs
import multiprocessing

In [46]:
gs_stores = pd.read_csv('gs://cmip6/cmip6-zarr-consolidated-stores.csv')
gcs = gcsfs.GCSFileSystem(token='anon')
gs_stores.loc[:,'ism'] = gs_stores.loc[:,'institution_id'] + '_' + gs_stores.loc[:,'source_id'] + '_' + gs_stores.loc[:,'member_id']

def get_annual_CMIP6_data_info(activity, table, variable, experiment, institution, source, member):
    # eg activity='CMIP', table='Amon', variable='tas', experiment='historical', institution="NCAR", source="CESM2", member="r10i1p1f1"
    
    query = gs_stores.query("activity_id==\'"+activity+"\' & table_id==\'"+table+"\' & variable_id==\'"+variable+"\' & experiment_id==\'"+experiment+"\' & institution_id==\'"+institution+"\' & source_id==\'"+source+"\' & member_id==\'"+member+"\'")
    
    if query.empty:
        print('No results for this request')
        return None

    # create a mutable-mapping-style interface to the store
    mapper = gcs.get_mapper(query.zstore.values[0])

    # open it using xarray and zarr
    ds = xr.open_zarr(mapper, consolidated=True)
    
    df  = pd.Series(name=institution+'_'+source+'_'+member+'_'+experiment,dtype=object)
    
    try:
        df.loc['parent_branch_time'] = ds.branch_time_in_parent
    except:
        df.loc['parent_branch_time'] = np.nan
    try:
        df.loc['parent_time_units'] = ds.parent_time_units
    except:
        df.loc['parent_time_units'] = np.nan
    try:
        df.loc['parent_variant'] = ds.parent_variant_label
    except:
        df.loc['parent_variant'] = np.nan
    try:
        df.loc['parent_experiment'] = ds.parent_experiment_id
    except:
        df.loc['parent_experiment'] = np.nan
    try:
        df.loc['parent_source'] = ds.parent_source_id
    except:
        df.loc['parent_source'] = np.nan
    try:
        df.loc['comment'] = ds.comment
    except:
        df.loc['comment'] = np.nan
    try:
        df.loc['calendar'] = ds.time.values[0].__class__
    except:
        df.loc['comment'] = np.nan  
        
    return df

In [47]:
all_info = gs_stores.loc[(gs_stores.experiment_id.isin(['1pctCO2','abrupt-4xCO2']))&(gs_stores.variable_id.isin(['tas']))&(gs_stores.table_id=='Amon')]

all_info_data = []

for index,row in all_info.iterrows():
    print('getting '+row.ism)
    all_info_data += [get_annual_CMIP6_data_info(row.loc['activity_id'], row.loc['table_id'], row.loc['variable_id'], row.loc['experiment_id'], row.loc['institution_id'], row.loc['source_id'], row.loc['member_id'])]

getting AS-RCEC_TaiESM1_r1i1p1f1
getting AS-RCEC_TaiESM1_r1i1p1f1
getting AWI_AWI-CM-1-1-MR_r1i1p1f1
getting AWI_AWI-CM-1-1-MR_r1i1p1f1
getting BCC_BCC-CSM2-MR_r1i1p1f1
getting BCC_BCC-CSM2-MR_r1i1p1f1
getting BCC_BCC-ESM1_r1i1p1f1
getting BCC_BCC-ESM1_r1i1p1f1
getting CAMS_CAMS-CSM1-0_r1i1p1f1
getting CAMS_CAMS-CSM1-0_r2i1p1f1
getting CAMS_CAMS-CSM1-0_r1i1p1f1
getting CAMS_CAMS-CSM1-0_r2i1p1f1
getting CAS_CAS-ESM2-0_r1i1p1f1
getting CAS_CAS-ESM2-0_r1i1p1f1
getting CAS_FGOALS-f3-L_r1i1p1f1
getting CAS_FGOALS-f3-L_r2i1p1f1
getting CAS_FGOALS-f3-L_r3i1p1f1
getting CAS_FGOALS-f3-L_r1i1p1f1
getting CAS_FGOALS-f3-L_r2i1p1f1
getting CAS_FGOALS-f3-L_r3i1p1f1
getting CAS_FGOALS-g3_r1i1p1f1
getting CAS_FGOALS-g3_r2i1p1f1
getting CAS_FGOALS-g3_r3i1p1f1
getting CAS_FGOALS-g3_r1i1p1f1
getting CCCma_CanESM5-CanOE_r1i1p2f1
getting CCCma_CanESM5_r1i1p1f1
getting CCCma_CanESM5_r1i1p2f1
getting CCCma_CanESM5_r2i1p1f1
getting CCCma_CanESM5_r2i1p2f1
getting CCCma_CanESM5_r3i1p1f1
getting CCCma_CanESM5_r3

/home/leachl/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/leachl/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


getting CSIRO-ARCCSS_ACCESS-CM2_r1i1p1f1
getting CSIRO_ACCESS-ESM1-5_r1i1p1f1
getting CSIRO_ACCESS-ESM1-5_r1i1p1f1
getting E3SM-Project_E3SM-1-0_r1i1p1f1
getting E3SM-Project_E3SM-1-0_r1i1p1f1
getting EC-Earth-Consortium_EC-Earth3-Veg_r1i1p1f1
getting EC-Earth-Consortium_EC-Earth3-Veg_r1i1p1f1
getting EC-Earth-Consortium_EC-Earth3_r8i1p1f1
getting FIO-QLNM_FIO-ESM-2-0_r1i1p1f1
getting FIO-QLNM_FIO-ESM-2-0_r2i1p1f1
getting FIO-QLNM_FIO-ESM-2-0_r3i1p1f1
getting FIO-QLNM_FIO-ESM-2-0_r1i1p1f1
getting FIO-QLNM_FIO-ESM-2-0_r2i1p1f1
getting FIO-QLNM_FIO-ESM-2-0_r3i1p1f1
getting HAMMOZ-Consortium_MPI-ESM-1-2-HAM_r1i1p1f1
getting HAMMOZ-Consortium_MPI-ESM-1-2-HAM_r1i1p1f1
getting INM_INM-CM4-8_r1i1p1f1
getting INM_INM-CM4-8_r1i1p1f1
getting INM_INM-CM5-0_r1i1p1f1
getting INM_INM-CM5-0_r1i1p1f1
getting IPSL_IPSL-CM6A-LR_r1i1p1f1
getting IPSL_IPSL-CM6A-LR_r10i1p1f1
getting IPSL_IPSL-CM6A-LR_r11i1p1f1
getting IPSL_IPSL-CM6A-LR_r12i1p1f1
getting IPSL_IPSL-CM6A-LR_r1i1p1f1
getting IPSL_IPSL-CM6A-LR_

/home/leachl/miniconda3/lib/python3.7/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/leachl/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


getting MIROC_MIROC6_r1i1p1f1
getting MOHC_HadGEM3-GC31-LL_r1i1p1f3
getting MOHC_HadGEM3-GC31-LL_r2i1p1f3
getting MOHC_HadGEM3-GC31-LL_r3i1p1f3
getting MOHC_HadGEM3-GC31-LL_r4i1p1f3
getting MOHC_HadGEM3-GC31-LL_r1i1p1f3
getting MOHC_HadGEM3-GC31-MM_r1i1p1f3
getting MOHC_HadGEM3-GC31-MM_r1i1p1f3
getting MOHC_UKESM1-0-LL_r1i1p1f2
getting MOHC_UKESM1-0-LL_r2i1p1f2
getting MOHC_UKESM1-0-LL_r3i1p1f2
getting MOHC_UKESM1-0-LL_r4i1p1f2
getting MOHC_UKESM1-0-LL_r1i1p1f2
getting MPI-M_MPI-ESM1-2-HR_r1i1p1f1
getting MPI-M_MPI-ESM1-2-HR_r1i1p1f1
getting MPI-M_MPI-ESM1-2-LR_r1i1p1f1
getting MPI-M_MPI-ESM1-2-LR_r1i1p1f1
getting MRI_MRI-ESM2-0_r1i1p1f1
getting MRI_MRI-ESM2-0_r1i2p1f1
getting MRI_MRI-ESM2-0_r10i1p1f1
getting MRI_MRI-ESM2-0_r11i1p1f1
getting MRI_MRI-ESM2-0_r12i1p1f1
getting MRI_MRI-ESM2-0_r13i1p1f1
getting MRI_MRI-ESM2-0_r1i1p1f1
getting MRI_MRI-ESM2-0_r1i2p1f1
getting MRI_MRI-ESM2-0_r2i1p1f1
getting MRI_MRI-ESM2-0_r3i1p1f1
getting MRI_MRI-ESM2-0_r4i1p1f1
getting MRI_MRI-ESM2-0_r5i1p1f

In [48]:
pd.concat(all_info_data,axis=1).to_csv('./cmip6_data/cmip6_branch_info.csv')